In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi
from importlib import reload

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

fileR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_enant_scan25step.root")
fileS = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_enant_scan25step.root")

PATH_CH = ["angular_distr_el/CH11-2-3_m2x-m29/","angular_distr_el/CH9-10_m1x-m42/"]
DIR = ["map_AVG_ph_En/","map_ScanEn_SHORT/","map_ScanEn_LONG/"]
EN = ["NO_gate/","700/","350/","20/"]
# L or R
ID_AVG=["PECD_el(theta)","PECD_el(theta)_redPHI"]
ID_SHORT=["cos(theta)_vs_pEn","cos(theta)_vs_pEn_pol","cos(theta)_vs_pEn_redPHI","cos(theta)_vs_pEn_pol_redPHI"]
ID_PECD=["PECD_el(theta)_@534.5eV_MINUS","PECD_el(theta)_@535.5eV_PLUS","PECD_el(theta)_@534.5eV_MINUS_redPHI","PECD_el(theta)_@535.5eV_PLUS_redPHI"]

loc_avg=PATH_CH[0]+DIR[0]+EN[0]
loc_short=PATH_CH[0]+DIR[1]+EN[0]
loc_long=PATH_CH[0]+DIR[2]+EN[0]

In [ ]:
fileR[loc_short+"R/"+ID_SHORT[0]].all_members
# fileR[loc_short+"R/"+ID_SHORT[0]].member("fXaxis") #just the object
fileR[loc_short+"R/"+ID_SHORT[0]].axis("x").centers() #N
# fileR[loc_short+"R/"+ID_SHORT[0]].axis("x").edges() #N+1
# fileR[loc_short+"R/"+ID_SHORT[0]].to_numpy()

In [ ]:
#load single graphs
en=fileR[loc_short+"R/"+ID_SHORT[0]].axis("x").centers() #N
ctheta=fileR[loc_short+"R/"+ID_SHORT[0]].axis("y").centers() #N

cthetaEN_R=fileR[loc_short+"R/"+ID_SHORT[0]].to_numpy()[0]
cthetaEN_R_err=fileR[loc_short+"R/"+ID_SHORT[0]].errors() #to be normalised too
cthetaEN_R_norm,cthetaEN_R_err_norm=fugi.normalise_with_err(cthetaEN_R,cthetaEN_R_err,normtype=0)

cthetaEN_L=fileR[loc_short+"L/"+ID_SHORT[0]].to_numpy()[0]
cthetaEN_L_err=fileR[loc_short+"L/"+ID_SHORT[0]].errors() #to be normalised too
cthetaEN_L_norm,cthetaEN_L_err_norm=fugi.normalise_with_err(cthetaEN_L,cthetaEN_L_err,normtype=0)

print(cthetaEN_R.shape)

In [ ]:
#PECD between two diffrent helicities
cPECD_en =  np.divide(np.subtract(cthetaEN_R_norm[3:],cthetaEN_L_norm[3:]),np.add(cthetaEN_R_norm[3:],cthetaEN_L_norm[3:]))
errPECD_en = fugi.error_calc(cthetaEN_R_norm[3:],cthetaEN_L_norm[3:],cthetaEN_R_err_norm[3:],cthetaEN_L_err_norm[3:])

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
cs1=ax1.contourf(en[3:],ctheta,cthetaEN_R_norm[3:].T, cmap=cmap_temp, levels=15)
ax1.set_title("ctehta vs EN R")
ax1.set_xlabel('en [eV]')
ax1.set_ylabel('cos(theta) [adm]')
# ax1.set_aspect(aspect)
cbar1=fig.colorbar(cs1,extend='both', shrink=0.9, ax=ax1)
cbar1.set_ticklabels(["min","max"])
cbar1.set_ticks([cthetaEN_R_norm[3:].min(),cthetaEN_R_norm[3:].max()])


cs2=ax2.contourf(en[3:],ctheta,cthetaEN_L_norm[3:].T, cmap=cmap_temp, levels=15)
ax2.set_title("ctehta vs EN L")
ax2.set_xlabel('en [eV]')
ax2.set_ylabel('cos(theta) [adm]')
# ax2.set_aspect(aspect)
cbar2=fig.colorbar(cs2,extend='both', shrink=0.9, ax=ax2)
cbar2.set_ticklabels(["min","max"])
cbar2.set_ticks([cthetaEN_L_norm[3:].min(),cthetaEN_L_norm[3:].max()])

# fig.savefig("../PYTHON_graphs/OUTPUTS/RCX_MFPAD_norm0_comparison_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)
# plt.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(en[3:],ctheta,cPECD_en.T, cmap=cmap_temp, levels=20)
ax.set_title("PECD vs energy")
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([cPECD_en.min(),cPECD_en.max()])
cbar.ax.set_ylabel('% of asymmetry')
# ax.set_aspect(aspect)
# fig.savefig("../PYTHON_graphs/OUTPUTS/SCR_MFPAD_singleNONORM_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
import functions_minuit as fumi
# param_matrix=np.zeros((6,72)) # alternative nparray initialization
param_matrix_cos = [[0 for i in range(6)] for j in range(21)] #i col, j row

# fig, axes = plt.subplots(3,7, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(3,7, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.1,0.1) #custom_ylim = (-1,1)

# limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]

for i,el,el_err,ax in zip(range(21),cPECD_en,errPECD_en,axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    
    x_data = ctheta;y_data = el;y_err = el_err
    #non sorted as test
    # x_data = ctheta_axis_red;y_data = cPECD_std[i];y_err = errPECD_std[i]
    
    m=fumi.iminuit_err(x_data,y_data,y_err,limits)
    fumi.plotminuit(x_data,y_data,y_err,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_cos[i][counter]=((m.values[el], m.errors[el]))

param_matrix_cos=np.array(param_matrix_cos) #numpy tranformation!!!

fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
# fig.savefig("../PYTHON_graphs/OUTPUTS/b1matrix_S_CR-CL_cos_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)
plt.show() 

In [ ]:
plt.scatter(en[3:],param_matrix_cos[:,0,0])


In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
cs1=ax1.scatter(en[3:],ctheta,param_matrix_cos[:,0,0], cmap=cmap_temp, levels=15)
ax1.set_title("ctehta vs EN R")
ax1.set_xlabel('en [eV]')
ax1.set_ylabel('cos(theta) [adm]')
# ax1.set_aspect(aspect)
cbar1=fig.colorbar(cs1,extend='both', shrink=0.9, ax=ax1)
cbar1.set_ticklabels(["min","max"])
cbar1.set_ticks([cthetaEN_R_norm[3:].min(),cthetaEN_R_norm[3:].max()])


cs2=ax2.contourf(en[3:],ctheta,cthetaEN_L_norm[3:].T, cmap=cmap_temp, levels=15)
ax2.set_title("ctehta vs EN L")
ax2.set_xlabel('en [eV]')
ax2.set_ylabel('cos(theta) [adm]')
# ax2.set_aspect(aspect)
cbar2=fig.colorbar(cs2,extend='both', shrink=0.9, ax=ax2)
cbar2.set_ticklabels(["min","max"])
cbar2.set_ticks([cthetaEN_L_norm[3:].min(),cthetaEN_L_norm[3:].max()])

# fig.savefig("../PYTHON_graphs/OUTPUTS/RCX_MFPAD_norm0_comparison_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)
# plt.show()